# Checking schedules

This notebook contains:
1. Checking that the schedules submitted correctly via different flows (workflow, cli and run_function).
2. The access key of the schedule is not exposed.
3. Overwriting workflow scheduling works.

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import mlrun
import os

In [ ]:
run_db = mlrun.get_run_db()

## Loading project with workflows

In [ ]:
PROJECT_NAME = "schedule-test"

In [ ]:
project = mlrun.load_project(f"./{PROJECT_NAME}", "git://github.com/mlrun/project-demo.git", PROJECT_NAME, allow_cross_project=True)

In [ ]:
def assert_schedule_access_key_is_not_exposed(name: str):
    schedule = run_db.api_call("GET", f"projects/{project.name}/schedules/{name}", params={"include-credentials": True}).json()
    assert schedule["credentials"]["access_key"].startswith("$ref:mlrun-auth-secrets.")
    print("Pass!")

def assert_schedule_overwritten(name: str, schedule: str):
    scheduled_job = run_db.get_schedule(project.name, name)
    assert scheduled_job.scheduled_object["schedule"] == schedule
    print("Pass!")
    

## Running the workflow

In [ ]:
out = os.system(f"cd ./{PROJECT_NAME}; git config --global --add safe.directory {os.path.abspath(PROJECT_NAME)}")

In [ ]:
project.run("main", schedule="*/10 * * * *")

In [ ]:
if not mlrun.mlconf.is_ce_mode():
    assert_schedule_access_key_is_not_exposed("main")

## Overwriting a schedule 

In [ ]:
project.run("main", schedule="*/20 * * * *", dirty=True)

In [ ]:
if not mlrun.mlconf.is_ce_mode():
    assert_schedule_access_key_is_not_exposed("main")
assert_schedule_overwritten("main", "*/20 * * * *")

### Check from CLI

In [ ]:
out = os.popen(f"mlrun project ./{PROJECT_NAME} -r main -d --schedule '*/15 * * * *'").read()
print(out)

In [ ]:
if not mlrun.mlconf.is_ce_mode():
    assert_schedule_access_key_is_not_exposed("main")
assert_schedule_overwritten("main", "*/15 * * * *")

## Check function scheduling

In [ ]:
mlrun.run_function("prep-data", schedule="*/10 * * * *", name="data-preparation")

In [ ]:
if not mlrun.mlconf.is_ce_mode():
    assert_schedule_access_key_is_not_exposed("data-preparation")